In [1]:
from gurobipy import GRB
from models.SSPMF import SSPMFModel
from models.JGSMF import JGSMFModel
import pandas as pd
import os

def retrieve_format_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        
        first_row = lines[0].strip().split()
        num_jobs = int(first_row[0])
        num_tools = int(first_row[1])
        magazine_capacity = int(first_row[2])
        
        job_tools = {job: [] for job in range(1, num_jobs + 1)}
        
        for tool_index, line in enumerate(lines[1:], start=1):
            tool_requirements = [int(x) for x in line.strip().split()]
            for job_index, requirement in enumerate(tool_requirements, start=1):
                if requirement == 1:
                    job_tools[job_index].append(tool_index)
                    
    return num_jobs, num_tools, magazine_capacity, job_tools

directory = '../data/MTSP/Laporte/Tabela4/'
data = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        num_jobs, num_tools, magazine_capacity, job_tools = retrieve_format_data(file_path)
        data.append([filename, num_jobs, num_tools, magazine_capacity, job_tools])

df = pd.DataFrame(data, columns=['filename', 'num_jobs', 'num_tools', 'magazine_capacity', 'job_tools'])
#df = df.sample(frac=0.02).reset_index(drop=True)

Cliques: [[5, 1], [5, 2], [5, 3, 6], [5, 4]]
Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-23
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Fedora Linux 40 (Workstation Edition)")

CPU model: Intel(R) Core(TM) i5-8350U CPU @ 1.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 697 rows, 1254 columns and 3452 nonzeros
Model fingerprint: 0xcf491efe
Variable types: 0 continuous, 1254 integer (1254 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 512 rows and 1091 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Model is infeasible
No opti

In [5]:
file_path = '../data/MTSP/Laporte/Tabela4/L1-1.txt'
num_jobs, num_tools, magazine_capacity, job_tools = retrieve_format_data(file_path)

print("Number of Jobs:", num_jobs)
print("Number of Tools:", num_tools)
print("Magazine Capacity:", magazine_capacity)
print("Job Tool Requirements:", job_tools)

Number of Jobs: 9
Number of Tools: 15
Magazine Capacity: 5
Job Tool Requirements: {1: [10, 12], 2: [3, 8, 12, 13, 14], 3: [4, 10], 4: [6, 8, 9, 14, 15], 5: [4, 9, 15], 6: [2, 5, 7, 13, 15], 7: [11, 14], 8: [4, 5, 14], 9: [1, 9]}


In [6]:
jobs = [x for x in range(1, num_jobs + 1)]
tools = [x for x in range(1, num_tools + 1)]
print("Jobs:", jobs)
print("Tools:", tools)

Jobs: [1, 2, 3, 4, 5, 6, 7, 8, 9]
Tools: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [ ]:
for i, t in job_tools.items():
    if i == 4:
        print(t)

In [2]:
directory = '../data/MTSP/Laporte/Tabela4/'
data = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        num_jobs, num_tools, magazine_capacity, job_tools = retrieve_format_data(file_path)
        data.append([num_jobs, num_tools, magazine_capacity, job_tools])

df = pd.DataFrame(data, columns=['num_jobs', 'num_tools', 'magazine_capacity', 'job_tools'])
df = df.sample(frac=0.02).reset_index(drop=True)

In [ ]:
results = []

for index, row in df.iterrows():
    num_jobs = row['num_jobs']
    num_tools = row['num_tools']
    magazine_capacity = row['magazine_capacity']
    job_tools_requirements = row['job_tools']

    jobs = [x for x in range(1, num_jobs + 1)]
    tools = [x for x in range(1, num_tools + 1)]

    #print("Jobs: ", jobs)
    #print("Tools: ", tools)
    #print("Magazine Capacity: ", magazine_capacity)
    #print("Job Tools Requirements: ", job_tools_requirements)

    model = JGSMFModel(jobs, tools, magazine_capacity, job_tools_requirements)
    model.model.setParam('OutputFlag', 0) # Suppress Gurobi output
    model.model.setParam('TimeLimit', 60*60) # 1 hour time limit for each iteration

    print("Iteration number ", index)
    model.optimize()

    if model.model.status == GRB.OPTIMAL:
        solution_found = True
        solution = model.get_solution()
    else:
        solution_found = False
        solution = None

    runtime = model.model.Runtime

    results.append([num_jobs, num_tools, magazine_capacity, solution_found, solution, runtime])

results_df = pd.DataFrame(results, columns=['N', 'M', 'C', 'solution_found', 'solution', 'runtime'])
results_df
        



With original model

In [3]:
true_count = results_df['solution_found'].sum()
false_count = results_df.shape[0] - true_count

false_indices = results_df.index[results_df['solution_found'] == False].tolist()

print("True Count: ", true_count)
print("False Count: ", false_count)
print("False Indices: ", false_indices)

True Count:  330
False Count:  0
False Indices:  []


With "working" sspmf from new paper (in first sum of constraint 2i the upper limit was set to N-1 instead of N-2)

In [10]:
true_count = results_df['solution_found'].sum()
false_count = results_df.shape[0] - true_count

false_indices = results_df.index[results_df['solution_found'] == False].tolist()

print("True Count: ", true_count)
print("False Count: ", false_count)
print("False Indices: ", false_indices)

True Count:  299
False Count:  31
False Indices:  [18, 23, 25, 27, 28, 29, 82, 83, 84, 85, 86, 87, 88, 89, 161, 170, 175, 178, 180, 181, 183, 184, 186, 187, 188, 189, 194, 278, 289, 291, 318]


In [11]:
row_82 = df.iloc[82]
num_jobs = row_82['num_jobs']
num_tools = row_82['num_tools']
magazine_capacity = row_82['magazine_capacity']
job_tools_requirements = row_82['job_tools']
filename = row_82['filename']
print("Number of Jobs: ", num_jobs)
print("Number of Tools: ", num_tools)
print("Magazine Capacity: ", magazine_capacity)
print("Job Tools Requirements: ", job_tools_requirements)
print("Filename: ", filename)

Number of Jobs:  9
Number of Tools:  25
Magazine Capacity:  15
Job Tools Requirements:  {1: [1, 3, 4, 6, 7, 9, 10, 12, 13, 15, 17, 18, 20, 21, 25], 2: [3, 4, 5, 8, 12, 17, 18, 19, 21, 22, 23], 3: [4, 11, 12, 13, 14, 18, 20, 22, 23, 25], 4: [4, 5, 8, 10, 11, 12, 13, 15, 17, 19, 20, 23, 24], 5: [1, 2, 5, 6, 7, 14, 15, 16, 18, 21, 22, 24], 6: [2, 5, 7, 9, 10, 13, 14, 15, 16, 17, 19, 20, 24], 7: [3, 5, 6, 7, 9, 12, 15, 16, 17, 24], 8: [1, 3, 5, 6, 7, 8, 12, 13, 16, 17, 18, 21, 23, 24, 25], 9: [1, 4, 6, 7, 8, 9, 10, 11, 13, 14, 17, 19, 21, 22, 23]}
Filename:  L17-2.txt


In [4]:
job_nodes = list(range(1, 10+1))
print(job_nodes)
print("N: ", len(job_nodes))
print("N-1: ", len(job_nodes)-1)
print("N-2: ", len(job_nodes)-2)
for i in range(1, len(job_nodes)):
    print(i)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
N:  10
N-1:  9
N-2:  8
1
2
3
4
5
6
7
8
9


In [19]:
jobs = list(range(1, 10+1))
print(jobs)
num_nodes = list(range(0, len(jobs) + 3))
print(num_nodes)
job_nodes = list(range(1, len(jobs) + 1))
print(job_nodes)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [11]:
k = [1,2,3,4,5,6]
print(len(k))
print(list(range(0, len(k) + 3)))
print(list(range(1, len(k))))
START = 0
REPO = len(k) + 1
DETACHED = len(k) + 2
print(START)
print(REPO)
print(DETACHED)
print("----------------")
K = len(k) -1
print(K)
print(list(range(1, K)))

6
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3, 4, 5]
0
7
8
----------------
5
[1, 2, 3, 4]
